In [ ]:
# The code was removed by Watson Studio for sharing.

<bold>Capstone Project, Thomas Bettencourt</bold>


In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### This is for the "Segmenting and Clustering Neighborhoods in Toronto" assignment ### 


In the first step we get the data.

In [5]:
##We get the raw html convert to text and make a soup object.

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text


soup = BeautifulSoup(page, 'html.parser')




We convert just the table to a dataframe using the beatiful soup to find all html included in the table section. 

In [69]:

trnt = pd.read_html(str(soup.table))
trnt_df = pd.DataFrame(trnt[0])


tr_df.loc[tr_df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = tr_df.loc[tr_df['Neighbourhood'] == 'Not assigned', 'Borough']

tr_df = trnt_df[trnt_df.Borough != 'Not assigned']


#tr_df['Neighbourhood'].loc[tr_df['Postcode'] == 'M5A'] = 'Harbourfront, Regent Park'

tr_df['Neighbourhood'] = tr_df[['Postcode','Neighbourhood','Borough']].groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ','.join(x))
tr_df.drop_duplicates(inplace = True)



tr_df.shape



/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(103, 3)

Below we will import the csv with coordinates for the postal codes. We will attempt to use geocoder first.

In [70]:
df_coords = pd.read_csv('https://cocl.us/Geospatial_data')

!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [72]:
from geopy import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ImportError: cannot import name 'geocoder'

Geocoder has failed to load so we will use the data set given.

In [87]:



tr_df.set_index('Postal Code')
df_coords.set_index('Postal Code')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [90]:
tr_df.columns = ['Postal Code', 'Neighbourhood', 'Borough']
tc_df = tr_df.merge(df_coords, on = 'Postal Code', how = 'left')
#tc_df = pd.concat(tr_df, df_coords)
tc_df.head()



,Postal Code,Neighbourhood,Borough,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
